## 『本次練習內容』
#### 運用這幾天所學觀念搭建一個CNN分類器

## 『本次練習目的』
  #### 熟悉CNN分類器搭建步驟與原理
  #### 學員們可以嘗試不同搭法，如使用不同的Maxpooling層，用GlobalAveragePooling取代Flatten等等

In [1]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.datasets import cifar10
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder

Using TensorFlow backend.


In [22]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape) #(50000, 32, 32, 3)

## Normalize Data
def normalize(X_train, X_test):
        mean = np.mean(X_train, axis=(0,1,2,3))
        std = np.std(X_train, axis=(0, 1, 2, 3))
        X_train = (X_train - mean) / (std + 1e-7)
        X_test = (X_test - mean) / (std + 1e-7) 
        return X_train, X_test, mean, std

## Normalize Training and Testset    
x_train, x_test, mean_train, std_train = normalize(x_train, x_test) 

(50000, 32, 32, 3)


In [24]:
## OneHot Label 由(None, 1)-(None, 10)
## ex. label=2,變成[0,0,1,0,0,0,0,0,0,0]
one_hot = OneHotEncoder()
y_train = one_hot.fit_transform(y_train).toarray()
y_test = one_hot.transform(y_test).toarray()
y_train

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.]])

In [25]:

classifier=Sequential()

#卷積組合
classifier.add(Convolution2D(64, 3,
               padding='same',
               activation='relu',
               input_shape=(32, 32, 3)))#32,3,3,input_shape=(32,32,3),activation='relu''
classifier.add(BatchNormalization())

classifier.add(MaxPooling2D((2,2)))

#卷積組合
classifier.add(Convolution2D(128, 3,
               padding='same',
               activation='relu'))
classifier.add(BatchNormalization())

#flatten
classifier.add(Flatten())

#FC
classifier.add(Dense(256, activation='relu')) #output_dim=100,activation=relu

#輸出
classifier.add(Dense(10, activation='softmax'))

classifier.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_11 (Conv2D)           (None, 32, 32, 64)        1792      
_________________________________________________________________
batch_normalization_10 (Batc (None, 32, 32, 64)        256       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 16, 16, 128)       73856     
_________________________________________________________________
batch_normalization_11 (Batc (None, 16, 16, 128)       512       
_________________________________________________________________
flatten_6 (Flatten)          (None, 32768)             0         
_________________________________________________________________
dense_11 (Dense)             (None, 256)              

In [27]:
#超過兩個就要選categorical_crossentrophy
classifier.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
classifier.fit(x_train, y_train, batch_size=200, epochs=10)

Epoch 1/10
50000/50000 [==============================] - 143s 3ms/step - loss: 1.1292 - accuracy: 0.6073
Epoch 2/10
50000/50000 [==============================] - 147s 3ms/step - loss: 0.7393 - accuracy: 0.7422
Epoch 3/10
50000/50000 [==============================] - 151s 3ms/step - loss: 0.5129 - accuracy: 0.8210
Epoch 4/10
50000/50000 [==============================] - 151s 3ms/step - loss: 0.3233 - accuracy: 0.8893
Epoch 5/10
50000/50000 [==============================] - 151s 3ms/step - loss: 0.1833 - accuracy: 0.9386
Epoch 6/10
50000/50000 [==============================] - 151s 3ms/step - loss: 0.1159 - accuracy: 0.9620
Epoch 7/10
50000/50000 [==============================] - 151s 3ms/step - loss: 0.0788 - accuracy: 0.9747
Epoch 8/10
50000/50000 [==============================] - 146s 3ms/step - loss: 0.0798 - accuracy: 0.9736
Epoch 9/10
50000/50000 [==============================] - 147s 3ms/step - loss: 0.0979 - accuracy: 0.9661
Epoch 10/10
50000/50000 [=====================

## 預測新圖片，輸入影像前處理要與訓練時相同
#### ((X-mean)/(std+1e-7) ):這裡的mean跟std是訓練集的
## 維度如下方示範

In [28]:
input_example = (np.zeros(shape=(1, 32, 32, 3)) - mean_train) / (std_train + 1e-7) 
classifier.predict(input_example)

array([[1.0948786e-04, 9.2349237e-06, 7.4550961e-03, 9.8357832e-01,
        4.9874798e-04, 1.7879282e-05, 3.0242879e-04, 9.0114054e-07,
        8.0279009e-03, 5.0291566e-09]], dtype=float32)